In [264]:
import numpy as np
import pandas as pd
import seaborn as sns

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [265]:
cc_df = pd.read_csv('/aiffel/data/fraud.csv')

# EDA

In [266]:
cc_df.duplicated().sum() # 중복 데이터 확인

0

In [267]:
cc_df.head()

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
1,2019-01-01 00:12:34,4956828990005111019,"fraud_Schultz, Simonis and Little",grocery_pos,44.71,Kenneth,Robinson,M,269 Sanchez Rapids,Elizabeth,NJ,7208,40.6747,-74.2239,124967,Operational researcher,1980-12-21,09eff9c806365e2a6be12c1bbab3d70e,1325376754,40.079588,-74.848087,0
2,2019-01-01 00:17:16,180048185037117,fraud_Kling-Grant,grocery_net,46.28,Mary,Wall,F,2481 Mills Lock,Plainfield,NJ,7060,40.6152,-74.4150,71485,Leisure centre manager,1974-07-19,19e23c6a300c774354417befe4f31f8c,1325377036,40.021888,-74.228188,0
3,2019-01-01 00:20:15,374930071163758,fraud_Deckow-O'Conner,grocery_pos,64.09,Daniel,Escobar,M,61390 Hayes Port,Romulus,MI,48174,42.2203,-83.3583,31515,Police officer,1971-11-05,6f363661ba6b55889e488dd178f2a0af,1325377215,42.360426,-83.552316,0
4,2019-01-01 00:23:41,2712209726293386,fraud_Balistreri-Nader,misc_pos,25.58,Jenna,Brooks,F,50872 Alex Plain Suite 088,Baton Rouge,LA,70808,30.4066,-91.1468,378909,"Designer, furniture",1977-02-22,1654da2abfb9e79a5f99167fc9779558,1325377421,29.737426,-90.853194,0


In [268]:
cc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491134 entries, 0 to 491133
Data columns (total 22 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   trans_date_trans_time  491134 non-null  object 
 1   cc_num                 491134 non-null  int64  
 2   merchant               491134 non-null  object 
 3   category               491134 non-null  object 
 4   amt                    491134 non-null  float64
 5   first                  491134 non-null  object 
 6   last                   491134 non-null  object 
 7   gender                 491134 non-null  object 
 8   street                 491134 non-null  object 
 9   city                   491134 non-null  object 
 10  state                  491134 non-null  object 
 11  zip                    491134 non-null  int64  
 12  lat                    491134 non-null  float64
 13  long                   491134 non-null  float64
 14  city_pop               491134 non-nu

- NaN 값 존재하지 않음
- trans_date_trans_time, dob Column은 datetime으로 변환하는게 좋아보임

In [269]:
cc_df.describe()

,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
count,4.911340e+05,491134.000000,491134.000000,491134.000000,491134.000000,4.911340e+05,4.911340e+05,491134.000000,491134.000000,491134.000000
mean,3.706013e+17,69.050120,50770.532384,37.931230,-90.495619,1.213922e+05,1.358730e+09,37.930272,-90.495411,0.002533
std,1.260229e+18,160.322867,26854.947965,5.341193,12.990732,3.725751e+05,1.819402e+07,5.372986,13.004100,0.050264
min,5.038744e+11,1.000000,1843.000000,24.655700,-122.345600,4.600000e+01,1.325376e+09,23.655789,-123.345106,0.000000
25%,2.131124e+14,8.960000,28405.000000,33.746700,-97.235100,1.228000e+03,1.343087e+09,33.781388,-96.984814,0.000000
50%,3.531130e+15,42.170000,49628.000000,38.507200,-87.591700,5.760000e+03,1.357257e+09,38.545124,-87.573441,0.000000
75%,4.653879e+15,80.330000,75048.000000,41.520500,-80.731000,5.083500e+04,1.374626e+09,41.624294,-80.685567,0.000000
max,4.956829e+18,25086.940000,99323.000000,48.887800,-69.965600,2.906700e+06,1.388534e+09,49.887523,-68.965624,1.000000


- amt는 거래액수로 추정되며, 최솟값과 최댓값은 이해되는 수준이다
- 이외에 특이사항은 보이지 않는다

# Feature Engineering

In [270]:
cc_df.nunique()

trans_date_trans_time    488474
cc_num                      124
merchant                    693
category                     14
amt                       36446
first                        92
last                        102
gender                        2
street                      124
city                        121
state                        44
zip                         124
lat                         124
long                        124
city_pop                    122
job                         110
dob                         123
trans_num                491134
unix_time                488477
merch_lat                484503
merch_long               487445
is_fraud                      2
dtype: int64

- 보통 수치형 데이터는 모델링 적용 가능하고, 모델링 적용 후 별로 중요하지 않은 것이 확인되면 제외시킬 수 있음. 따라서 확실하게 지울 수 있는 것을 제외하고는 city pop 같은 애매한 column은 보류
- cc_num은 예측모델링엔 필요하진 않지만 feature engineering 에서 고객을 특정하기 위해 필요하므로 일단 유지
- 불필요한 식별자 first, last, trans_num 제거
- 상점 위치가 lat, long 형식으로 존재하므로, 이와 비교가능한 lat, long을 제외한 주소정보 street, city, state, zip 제거
- trans_date_trans_time이 존재하므로 unix_time 제거
- merchant의 경우 상점별 영향력 정보를 얻을 수 있겠으나 고유값 개수가 많고 category 존재하므로 제거
- job은 유의미할 수 있으나, 고유값이 많기도 하고 cc_num과 job의 고유값 개수가 별로 차이나지 않아 특정 직업별로 유의미한 정보를 얻기 어려워보이므로 여기서는 활용하지 않는다

In [271]:
cc_df.drop(['first', 'last', 'trans_num', 'street', 'city', 'state', 'zip', 'unix_time', 'merchant', 'job'], axis=1, inplace=True)

### 개인별 거래금액 확인

###### 개인별로 각 거래가 평소 거래 금액과 얼마나 차이가 나는지 확인해보자

In [272]:
cc_df.groupby('cc_num')['amt'].agg(['mean', 'std'])

,mean,std
cc_num,,
503874407318,60.253406,127.265783
567868110212,83.442558,117.303828
571365235126,59.392974,134.289959
581686439828,58.578675,149.804992
630423337322,56.078113,159.201852
...,...,...
4792627764422477317,84.135134,107.316736
4797297220948468262,56.313583,247.931817
4861310130652566408,85.805306,130.998089


In [273]:
amt_info = cc_df.groupby('cc_num')['amt'].agg(['mean', 'std']).reset_index()

- cc_df와 개인별 거래금액의 평균, 표준편차 데이터를 merge하기 위해 reset_index() 하여 저장

In [274]:
cc_df = cc_df.merge(amt_info, on='cc_num', how='left')

- cc_df의 데이터는 사라지면 안되므로 cc_num을 기준으로 cc_df와 amt_info left merge 수행

In [275]:
cc_df['amt_z'] = (cc_df['amt'] - cc_df['mean']) / cc_df['std']

- 각 거래에 대해 평소 거래와 얼마나 다른지 식별할 수 있는 z_score를 amt_z 컬럼에 저장

In [276]:
cc_df.head()

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,mean,std,amt_z
0,2019-01-01 00:00:44,630423337322,grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,56.078113,159.201852,0.321302
1,2019-01-01 00:12:34,4956828990005111019,grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,59.858059,132.138802,-0.114637
2,2019-01-01 00:17:16,180048185037117,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,87.328067,113.454416,-0.361802
3,2019-01-01 00:20:15,374930071163758,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,64.317839,174.739042,-0.001304
4,2019-01-01 00:23:41,2712209726293386,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,90.747123,165.470881,-0.393828


In [277]:
cc_df.loc[cc_df['is_fraud']==1]

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,mean,std,amt_z
4794,2019-01-12 00:59:01,581686439828,gas_transport,11.73,M,41.5205,-80.0573,5507,1973-07-28,41.947427,-79.796264,1,58.578675,149.804992,-0.312731
4816,2019-01-12 03:48:07,581686439828,grocery_pos,328.68,M,41.5205,-80.0573,5507,1973-07-28,42.148618,-79.398595,1,58.578675,149.804992,1.803020
4979,2019-01-12 15:46:10,581686439828,food_dining,120.58,M,41.5205,-80.0573,5507,1973-07-28,42.470024,-80.126576,1,58.578675,149.804992,0.413880
5073,2019-01-12 19:53:59,581686439828,shopping_net,1081.35,M,41.5205,-80.0573,5507,1973-07-28,42.455406,-79.521640,1,58.578675,149.804992,6.827351
5124,2019-01-12 22:44:05,581686439828,shopping_net,776.70,M,41.5205,-80.0573,5507,1973-07-28,40.680209,-79.099101,1,58.578675,149.804992,4.793708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477832,2020-12-21 02:21:41,4716561796955522,grocery_pos,358.24,F,48.2777,-112.8456,743,1972-05-04,47.526202,-113.643313,1,52.537867,106.113023,2.880911
477847,2020-12-21 02:36:03,4716561796955522,shopping_net,859.12,F,48.2777,-112.8456,743,1972-05-04,48.272348,-112.328075,1,52.537867,106.113023,7.601161
479296,2020-12-21 22:38:38,4716561796955522,home,209.84,F,48.2777,-112.8456,743,1972-05-04,49.173669,-112.698767,1,52.537867,106.113023,1.482402
479305,2020-12-21 22:42:11,4716561796955522,food_dining,123.58,F,48.2777,-112.8456,743,1972-05-04,48.913048,-113.214921,1,52.537867,106.113023,0.669495


- 사기 거래 데이터의 z-score를 확인해보니 점수값이 큰 거래도 있으나 정상범주의 거래금액도 몇몇 보임

In [278]:
cc_df.drop(['mean', 'std'], axis=1, inplace=True)

- z-score 계산 끝났으니 mean, std는 제거

### 거래 항목별 금액 확인

In [279]:
cc_df.groupby(['cc_num', 'category'])['amt'].agg(['mean', 'std'])

mean         std
cc_num              category                            
503874407318        entertainment  73.282418  103.050402
                    food_dining    38.712305   46.548436
                    gas_transport  68.457820   14.730440
                    grocery_net    48.931302   18.736252
                    grocery_pos    61.987806   23.449569
...                                      ...         ...
4956828990005111019 misc_pos       74.177012  168.341518
                    personal_care  35.379382   44.082579
                    shopping_net   70.019115  239.350164
                    shopping_pos   45.988976  174.986921
                    travel         72.701961  456.554619

[1736 rows x 2 columns]

In [280]:
cat_info = cc_df.groupby(['cc_num', 'category'])['amt'].agg(['mean', 'std']).reset_index()

- 거래 항목별 평균과 표준편차도 cc_df에 merge 하기 위해 reset_index() 사용해 저장

In [281]:
cc_df = cc_df.merge(cat_info, on=['cc_num', 'category'], how='left')

- 항목별 평균과 표준편차와 cc_df merge 수행

In [282]:
cc_df.head()

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,mean,std
0,2019-01-01 00:00:44,630423337322,grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.321302,99.637224,23.904424
1,2019-01-01 00:12:34,4956828990005111019,grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.114637,60.694144,24.513316
2,2019-01-01 00:17:16,180048185037117,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.361802,60.427269,19.558574
3,2019-01-01 00:20:15,374930071163758,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.001304,59.145831,23.345746
4,2019-01-01 00:23:41,2712209726293386,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.393828,86.794025,106.330185


In [283]:
cc_df['cat_amt_z'] = (cc_df['amt'] - cc_df['mean']) / cc_df['std']

- 항목별 금액에 대한 z-score계산

In [284]:
cc_df.head()

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,mean,std,cat_amt_z
0,2019-01-01 00:00:44,630423337322,grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.321302,99.637224,23.904424,0.317631
1,2019-01-01 00:12:34,4956828990005111019,grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.114637,60.694144,24.513316,-0.652060
2,2019-01-01 00:17:16,180048185037117,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.361802,60.427269,19.558574,-0.723328
3,2019-01-01 00:20:15,374930071163758,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.001304,59.145831,23.345746,0.211780
4,2019-01-01 00:23:41,2712209726293386,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.393828,86.794025,106.330185,-0.575698


In [285]:
cc_df.drop(['mean', 'std'], axis=1, inplace=True)

- z-score 계산 끝났으니 mean, std 제거

In [286]:
cc_df.head()

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,cat_amt_z
0,2019-01-01 00:00:44,630423337322,grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.321302,0.317631
1,2019-01-01 00:12:34,4956828990005111019,grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.114637,-0.652060
2,2019-01-01 00:17:16,180048185037117,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.361802,-0.723328
3,2019-01-01 00:20:15,374930071163758,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.001304,0.211780
4,2019-01-01 00:23:41,2712209726293386,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.393828,-0.575698


### 결제 시간 관련 분석

In [287]:
%%time
pd.to_datetime(cc_df['trans_date_trans_time']).map(lambda x: x.hour)

CPU times: user 1.33 s, sys: 41 ms, total: 1.37 s
Wall time: 1.37 s


0          0
1          0
2          0
3          0
4          0
          ..
491129    23
491130    23
491131    23
491132    23
491133    23
Name: trans_date_trans_time, Length: 491134, dtype: int64

In [288]:
%%time
pd.to_datetime(cc_df['trans_date_trans_time']).dt.hour

CPU times: user 110 ms, sys: 1.02 ms, total: 111 ms
Wall time: 109 ms


0          0
1          0
2          0
3          0
4          0
          ..
491129    23
491130    23
491131    23
491132    23
491133    23
Name: trans_date_trans_time, Length: 491134, dtype: int64

- dt.hour 로 계산하는게 map 사용보다 빠름 => dt 사용 권장

In [289]:
cc_df['hour'] = pd.to_datetime(cc_df['trans_date_trans_time']).dt.hour

- 거래 시간 hour 컬럼에 추가

In [290]:
cc_df.tail()

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,cat_amt_z,hour
491129,2020-12-31 23:56:48,6011109736646996,home,134.26,F,34.2651,-77.8670,186140,1983-02-08,34.853497,-78.664158,0,0.360311,1.450980,23
491130,2020-12-31 23:56:57,213112402583773,shopping_pos,25.49,F,34.0326,-82.2027,1523,1984-06-03,35.008839,-81.475156,0,-0.173577,-0.128877,23
491131,2020-12-31 23:59:09,3556613125071656,kids_pets,111.84,M,29.0393,-95.4401,28739,1999-12-27,29.661049,-96.186633,0,0.364619,1.163822,23
491132,2020-12-31 23:59:15,6011724471098086,kids_pets,86.88,F,46.1966,-118.9017,3684,1981-11-29,46.658340,-119.715054,0,-0.015209,0.440137,23
491133,2020-12-31 23:59:34,4170689372027579,entertainment,38.13,M,35.6665,-97.4798,116001,1993-05-10,36.210097,-97.036372,0,-0.152588,-0.380698,23


In [291]:
def hour_func(x):
    if (x >= 6) & (x < 12):
        return 'morning'
    elif (x >= 12) & (x < 18):
        return 'afternoon'
    elif (x >= 18) & (x < 23):
        return 'night'
    else:
        return 'evening'

- 결제시간 분류 함수 생성 (map과 lambda로는 이런 복잡한 함수 구현이 어려움)

In [292]:
cc_df['hour_cat'] = cc_df['hour'].map(hour_func)

- hour 컬럼에 hour_func 적용한 값 hour_cat에 저장

In [293]:
cc_df.head()

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,cat_amt_z,hour,hour_cat
0,2019-01-01 00:00:44,630423337322,grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.321302,0.317631,0,evening
1,2019-01-01 00:12:34,4956828990005111019,grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.114637,-0.652060,0,evening
2,2019-01-01 00:17:16,180048185037117,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.361802,-0.723328,0,evening
3,2019-01-01 00:20:15,374930071163758,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.001304,0.211780,0,evening
4,2019-01-01 00:23:41,2712209726293386,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.393828,-0.575698,0,evening


In [294]:
cc_df['hour_cat'].value_counts()

afternoon    176801
night        146697
evening       98662
morning       68974
Name: hour_cat, dtype: int64

- 시간대별 데이터 수 계산

In [295]:
cc_df.groupby('cc_num')['amt'].count()

cc_num
503874407318           3655
567868110212           3644
571365235126           4374
581686439828           3653
630423337322           4362
                       ... 
4792627764422477317    4357
4797297220948468262    3653
4861310130652566408    3643
4906628655840914250    3655
4956828990005111019    3657
Name: amt, Length: 124, dtype: int64

- cc_num별 amt 데이터 수 계산

In [296]:
all_cnt = cc_df.groupby('cc_num')['amt'].count().reset_index()

- cc_num을 기준으로 merge하기 위해 reset_index() 사용

In [297]:
cc_df.groupby(['cc_num', 'hour_cat'])['amt'].count()

cc_num               hour_cat 
503874407318         afternoon    1280
                     evening       737
                     morning       558
                     night        1080
567868110212         afternoon    1228
                                  ... 
4906628655840914250  night        1073
4956828990005111019  afternoon    1277
                     evening       784
                     morning       558
                     night        1038
Name: amt, Length: 496, dtype: int64

- 개개인마다 시간대별 amt데이터 수 계산

In [298]:
hour_cnt = cc_df.groupby(['cc_num', 'hour_cat'])['amt'].count().reset_index()

- cc_num, hour_cat을 기준으로 merge하기 위해 reset_index() 사용

In [299]:
hour_cnt = hour_cnt.merge(all_cnt, on='cc_num', how='left')

- cc_num을 기준으로 hour_cnt과 all_cnt left merge

In [300]:
hour_cnt.rename({'amt_x': 'hour_cnt', 'amt_y': 'total_cnt'}, axis=1, inplace=True)

- amt_x, amt_y의 구분이 어려우니 각각 hour_cnt, total_cnt로 이름 변경

In [301]:
hour_cnt.head()

,cc_num,hour_cat,hour_cnt,total_cnt
0,503874407318,afternoon,1280,3655
1,503874407318,evening,737,3655
2,503874407318,morning,558,3655
3,503874407318,night,1080,3655
4,567868110212,afternoon,1228,3644


In [302]:
hour_cnt['hour_perc'] = hour_cnt['hour_cnt'] / hour_cnt['total_cnt']

- 전체 거래 중, 시간대별 거래 비율이 얼마나 되는지 확인하기 위해 total_cnt에 대한 hour_cnt의 비율을 계산하여 hour_perc에 추가

In [303]:
hour_cnt.head()

,cc_num,hour_cat,hour_cnt,total_cnt,hour_perc
0,503874407318,afternoon,1280,3655,0.350205
1,503874407318,evening,737,3655,0.201642
2,503874407318,morning,558,3655,0.152668
3,503874407318,night,1080,3655,0.295486
4,567868110212,afternoon,1228,3644,0.336992


In [304]:
hour_cnt = hour_cnt[['cc_num', 'hour_cat', 'hour_perc']]

- hour_perc를 계산했으므로 hour_cnt, total_cnt는 제외

In [305]:
cc_df = cc_df.merge(hour_cnt, on=['cc_num', 'hour_cat'], how='left')

- cc_num, hour_cat을 기준으로 cc_df와 hour_cnt left merge

In [306]:
cc_df.drop(['trans_date_trans_time', 'hour', 'hour_cat'], axis=1, inplace=True)

- 시간 정보 추출이 완료되었으므로 trans_date_trans_time, hour, hour_cat 정보는 제거한다

### 거리 정보 분석

In [307]:
!pip install geopy

In [308]:
from geopy.distance import distance

- lat, long 컬럼으로 거리 계산을 하기 위해 geopy 라이브러리의 distance 함수 import

In [309]:
cc_df['distance'] = cc_df.apply(lambda x: distance((x['lat'], x['long']), (x['merch_lat'], x['merch_long'])).km, axis=1)

- 각 거래별 고객 위치와 상점 위치의 거리(km) 계산하여 distance에 추가
- 시리즈에 대한 사칙연산이 아님 -> distance((x['lat'], x['long']), (x['merch_lat'], x['merch_long'])).km 만으로 계산 불가
- series 대한 연산이 아님 -> apply 사용해 dataframe의 연산

In [310]:
cc_df.head()

,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,cat_amt_z,hour_perc,distance
0,630423337322,grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.321302,0.317631,0.196470,30.216618
1,4956828990005111019,grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.114637,-0.652060,0.214383,84.714605
2,180048185037117,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.361802,-0.723328,0.217252,67.768167
3,374930071163758,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.001304,0.211780,0.213600,22.322745
4,2712209726293386,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.393828,-0.575698,0.202882,79.398244


In [311]:
cc_df.groupby('cc_num')['distance'].agg(['mean', 'std'])

,mean,std
cc_num,,
503874407318,79.746738,29.741044
567868110212,79.356969,30.524201
571365235126,73.651715,28.863007
581686439828,74.428143,28.605102
630423337322,71.656621,28.090646
...,...,...
4792627764422477317,75.499945,29.146824
4797297220948468262,75.630035,29.148428
4861310130652566408,78.977581,29.793035


In [312]:
dist_info = cc_df.groupby('cc_num')['distance'].agg(['mean', 'std']).reset_index()

- 각 고객마다 distance의 평균과 표준편차를 계산하여 dist_info에 저장
- cc_df와 cc_num 기준으로 merge하기 위해 reset_index() 사용

In [313]:
cc_df = cc_df.merge(dist_info, on='cc_num', how='left')

- cc_num을 기준으로 cc_df와 dist_info merge

In [314]:
cc_df.head()

,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,cat_amt_z,hour_perc,distance,mean,std
0,630423337322,grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.321302,0.317631,0.196470,30.216618,71.656621,28.090646
1,4956828990005111019,grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.114637,-0.652060,0.214383,84.714605,74.811123,28.675031
2,180048185037117,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.361802,-0.723328,0.217252,67.768167,75.617531,28.784606
3,374930071163758,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.001304,0.211780,0.213600,22.322745,74.706461,28.711493
4,2712209726293386,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.393828,-0.575698,0.202882,79.398244,79.334924,29.620117


In [315]:
cc_df['dist_z'] = (cc_df['distance'] - cc_df['mean']) / cc_df['std']

- 각 거래별 distance의 z-score 계산하여 dist_z에 저장

In [316]:
cc_df.head()

,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,cat_amt_z,hour_perc,distance,mean,std,dist_z
0,630423337322,grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.321302,0.317631,0.196470,30.216618,71.656621,28.090646,-1.475224
1,4956828990005111019,grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.114637,-0.652060,0.214383,84.714605,74.811123,28.675031,0.345370
2,180048185037117,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.361802,-0.723328,0.217252,67.768167,75.617531,28.784606,-0.272693
3,374930071163758,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.001304,0.211780,0.213600,22.322745,74.706461,28.711493,-1.824486
4,2712209726293386,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.393828,-0.575698,0.202882,79.398244,79.334924,29.620117,0.002138


In [317]:
cc_df.drop(['lat', 'long', 'merch_lat', 'merch_long', 'mean', 'std'], axis=1, inplace=True)

- 거리 정보를 활용해 z-score 계산했으니, distance와 dist-z만 남겨두고 나머지 거리 관련 column은 제거

In [318]:
cc_df.head()

,cc_num,category,amt,gender,city_pop,dob,is_fraud,amt_z,cat_amt_z,hour_perc,distance,dist_z
0,630423337322,grocery_pos,107.23,F,149,1978-06-21,0,0.321302,0.317631,0.196470,30.216618,-1.475224
1,4956828990005111019,grocery_pos,44.71,M,124967,1980-12-21,0,-0.114637,-0.652060,0.214383,84.714605,0.345370
2,180048185037117,grocery_net,46.28,F,71485,1974-07-19,0,-0.361802,-0.723328,0.217252,67.768167,-0.272693
3,374930071163758,grocery_pos,64.09,M,31515,1971-11-05,0,-0.001304,0.211780,0.213600,22.322745,-1.824486
4,2712209726293386,misc_pos,25.58,F,378909,1977-02-22,0,-0.393828,-0.575698,0.202882,79.398244,0.002138


### 출생 연도 column 추가

###### 방법 3

In [319]:
%%time
pd.to_datetime(cc_df['dob']).dt.year

CPU times: user 123 ms, sys: 1.95 ms, total: 125 ms
Wall time: 124 ms


0         1978
1         1980
2         1974
3         1971
4         1977
          ... 
491129    1983
491130    1984
491131    1999
491132    1981
491133    1993
Name: dob, Length: 491134, dtype: int64

###### 방법 2

In [320]:
%%time
cc_df['dob'].map(lambda x: x.split('-')[0]).astype(int)

CPU times: user 261 ms, sys: 36.1 ms, total: 297 ms
Wall time: 296 ms


0         1978
1         1980
2         1974
3         1971
4         1977
          ... 
491129    1983
491130    1984
491131    1999
491132    1981
491133    1993
Name: dob, Length: 491134, dtype: int64

###### 방법 3

In [321]:
%%time
cc_df['dob'].str.split('-', expand=True)[0].astype(int)

CPU times: user 2.12 s, sys: 194 ms, total: 2.31 s
Wall time: 2.3 s


0         1978
1         1980
2         1974
3         1971
4         1977
          ... 
491129    1983
491130    1984
491131    1999
491132    1981
491133    1993
Name: 0, Length: 491134, dtype: int64

In [322]:
cc_df['dob'] = pd.to_datetime(cc_df['dob']).dt.year

In [323]:
cc_df.head()

,cc_num,category,amt,gender,city_pop,dob,is_fraud,amt_z,cat_amt_z,hour_perc,distance,dist_z
0,630423337322,grocery_pos,107.23,F,149,1978,0,0.321302,0.317631,0.196470,30.216618,-1.475224
1,4956828990005111019,grocery_pos,44.71,M,124967,1980,0,-0.114637,-0.652060,0.214383,84.714605,0.345370
2,180048185037117,grocery_net,46.28,F,71485,1974,0,-0.361802,-0.723328,0.217252,67.768167,-0.272693
3,374930071163758,grocery_pos,64.09,M,31515,1971,0,-0.001304,0.211780,0.213600,22.322745,-1.824486
4,2712209726293386,misc_pos,25.58,F,378909,1977,0,-0.393828,-0.575698,0.202882,79.398244,0.002138


### One-hot encoding

In [324]:
cc_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 491134 entries, 0 to 491133
Data columns (total 12 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   cc_num     491134 non-null  int64  
 1   category   491134 non-null  object 
 2   amt        491134 non-null  float64
 3   gender     491134 non-null  object 
 4   city_pop   491134 non-null  int64  
 5   dob        491134 non-null  int64  
 6   is_fraud   491134 non-null  int64  
 7   amt_z      491134 non-null  float64
 8   cat_amt_z  491134 non-null  float64
 9   hour_perc  491134 non-null  float64
 10  distance   491134 non-null  float64
 11  dist_z     491134 non-null  float64
dtypes: float64(6), int64(4), object(2)
memory usage: 48.7+ MB


- gender는 M, F로 2가지의 값만 존재
- category의 고유값 확인 필요

In [325]:
cc_df['category'].nunique()

14

In [326]:
cc_df['category'].value_counts()

shopping_pos      48345
gas_transport     46029
home              43340
grocery_pos       42722
kids_pets         40295
shopping_net      39179
food_dining       35980
personal_care     35513
entertainment     33619
misc_pos          32411
health_fitness    31415
misc_net          25179
grocery_net       24486
travel            12621
Name: category, dtype: int64

- 각 value별로 10000개 이상의 데이터 가지고 있으므로 따로 합치진 않는다

In [327]:
cc_df = pd.get_dummies(cc_df, drop_first=True)

In [328]:
cc_df.head()

,cc_num,amt,city_pop,dob,is_fraud,amt_z,cat_amt_z,hour_perc,distance,dist_z,category_food_dining,category_gas_transport,category_grocery_net,category_grocery_pos,category_health_fitness,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel,gender_M
0,630423337322,107.23,149,1978,0,0.321302,0.317631,0.196470,30.216618,-1.475224,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,4956828990005111019,44.71,124967,1980,0,-0.114637,-0.652060,0.214383,84.714605,0.345370,0,0,0,1,0,0,0,0,0,0,0,0,0,1
2,180048185037117,46.28,71485,1974,0,-0.361802,-0.723328,0.217252,67.768167,-0.272693,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,374930071163758,64.09,31515,1971,0,-0.001304,0.211780,0.213600,22.322745,-1.824486,0,0,0,1,0,0,0,0,0,0,0,0,0,1
4,2712209726293386,25.58,378909,1977,0,-0.393828,-0.575698,0.202882,79.398244,0.002138,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [329]:
cc_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 491134 entries, 0 to 491133
Data columns (total 24 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   cc_num                   491134 non-null  int64  
 1   amt                      491134 non-null  float64
 2   city_pop                 491134 non-null  int64  
 3   dob                      491134 non-null  int64  
 4   is_fraud                 491134 non-null  int64  
 5   amt_z                    491134 non-null  float64
 6   cat_amt_z                491134 non-null  float64
 7   hour_perc                491134 non-null  float64
 8   distance                 491134 non-null  float64
 9   dist_z                   491134 non-null  float64
 10  category_food_dining     491134 non-null  uint8  
 11  category_gas_transport   491134 non-null  uint8  
 12  category_grocery_net     491134 non-null  uint8  
 13  category_grocery_pos     491134 non-null  uint8  
 14  cate

In [330]:
cc_df.drop('cc_num', axis=1, inplace=True)

- 모든 column의 정보 추출이 끝났고, 더이상의 식별자는 필요하지 않아 cc_num은 제거